In [ ]:
%matplotlib inline
import mne
#mne.viz.set_browser_backend('qt')


# The ModKmeans object

This tutorial introduces the :class:`pycrostates.clustering.ModKMeans`
structure in detail.


In [ ]:
from mne.io import read_raw_eeglab

from pycrostates.datasets import lemon


raw_fname = lemon.data_path(subject_id='010017', condition='EC')
raw = read_raw_eeglab(raw_fname, preload=True)
raw.crop(0, 30)

raw.pick('eeg')
raw.set_eeg_reference('average')

In [ ]:
raw.plot();

The modified Kmeans can be instantiated with the number of cluster centers
``n_clusters`` to compute.
A random_state can be defined during class definition in order to have
reproducible results.



In [ ]:
from pycrostates.cluster import ModKMeans

n_clusters = 5
ModK = ModKMeans(n_clusters=n_clusters, random_state=42)

Most methods need the modified Kmeans to be fitted. This can be done with
either `mne.io.Raw` or `mne.epochs.Epochs` data structures.
Note that, depending on your setup, you can change ``n_jobs=1`` in order to
use parallel processing and reduce computation time.



In [ ]:
ModK.fit(raw, n_jobs=5)

Now that our algorithm is fitted, we can visualize the cluster centers, also
called microstate maps or microstate topographies using `ModK.plot`.
Note than this method uses the `~mne.Info` object of the fitted
instance to display the topographies.



In [ ]:
ModK.plot();

One can access the cluster centers as a numpy array thanks to the
``cluster_centers_`` attribute:



In [ ]:
ModK.cluster_centers_

## Preprocessing

In [ ]:
from pycrostates.preprocessing import extract_gfp_peaks
gfp_data = extract_gfp_peaks(raw, min_peak_distance=3)
gfp_data

In [ ]:
ModK_ = ModKMeans(n_clusters=n_clusters, random_state=42)
ModK_.fit(gfp_data, n_jobs=5)
ModK_.plot(); # gfp
ModK.plot(); #all data

## Optimal number of maps

In [ ]:
from pycrostates.metrics import (
    silhouette_score,
    calinski_harabasz_score,
    dunn_score,
    davies_bouldin_score,
)

cluster_numbers = range(2,8)
silhouette_scores = []
calinski_harabasz_scores = []
dunn_scores = []
davies_bouldin_scores = []

for k in cluster_numbers:
    ModK_ = ModKMeans(n_clusters=k, random_state=42)
    ModK_.fit(raw, n_jobs=4)
    # silhouettee
    silhouette_scores.append(silhouette_score(ModK))
    # calinski and harabasz
    #calinski_harabasz_scores.append(calinski_harabasz_score(ModK))
    # dunn
    #dunn_scores.append(dunn_score(ModK))
    # davies bouldin
    #davies_bouldin_scores.append(davies_bouldin_score(ModK))

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.scatter(cluster_numbers, silhouette_scores)
plt.xlabel('n_clusters')
plt.ylabel('Silhouette score')
plt.show()

## Segmentation

Clusters centers can be reordered using `ModK.reorder_clusters`:



In [ ]:
ModK.reorder_clusters(order=[3, 2, 4, 0, 1])
ModK.plot();

and renamed using `ModK.rename_clusters`:



In [ ]:
ModK.rename_clusters(new_names=['A', 'B', 'C', 'D', 'F'])
ModK.plot();

Maps polarities can be inverted thanks to `ModK.invert_polarity`
method. Note that it only affects visualization, it has not effect during
backfitting as polarities are ignored.



In [ ]:
ModK.invert_polarity([False, False, True, True, False])
ModK.plot();

Finally, the modified Kmeans can be used to predict the microstates
segmentation using the `ModK.predict` method. By default, segments
annotated as bad will not be labeled, but this behavior can be changed with
the ``reject_by_annotation`` argument. Smoothing can be performed on the
output sequence by setting the ``factor`` argument ``> 0`` (no smoothing by
default ``factor=0``) while the ``half_window_size`` parameter is used to
specify the smoothing temporal span. Finally, the ``reject_edges`` argument
allows not to assign the first and last segment of each record (or each
epoch) as these can be incomplete. It should have little impact for raw, but
can be important when working with epochs.



In [ ]:
segmentation = ModK.predict(raw, reject_by_annotation=True, factor=10,
                            half_window_size=10, min_segment_length=5,
                            reject_edges=True)
segmentation.plot(tmin=1, tmax=5);

In [ ]:
segmentation.compute_parameters()